In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import scanpy.external as sce
import scanpy.logging as logg
import scvelo as scv
import matplotlib.pyplot as plt
import seaborn as sb
import gseapy as gp
import plotly.graph_objects as go
import scipy
scv.settings.presenter_view = True  # set max width size for presenter view
scv.settings.set_figure_params('scvelo')  # for beautified visualization
sc.settings.verbosity = 3
# Matplotlib backwards compatibility hack
import matplotlib
matplotlib.cbook.iterable = np.iterable

from IPython.display import display
import vdom.helpers as vh

import cellicium.develop as cdev
import cellicium.sharedata as cdata
import cellicium.scrna as crna
import cellicium.gene_sets as cgs
# cdev.reload_user_libs(cdata)
# cdev.reload_user_libs(crna)
# cdev.reload_user_libs(crna.tools)
# cdev.reload_user_libs(crna.qc)
# cdev.reload_user_libs(cgs)
gsm = cgs.GeneSetManager()

## Dataset


Source: 
- url: https://www.embopress.org/doi/full/10.15252/msb.20209946
- title: The transcriptome dynamics of single cells during the cell cycle
- authors: Daniel Schwabe, Sara Formichetti, Jan Philipp Junker, Martin Falcke, Nikolaus Rajewsky

Data: GSE142277
- Location: GSE142277/GSM4224315/GSM4224315_out_gene_exon_tagged.dge_exonssf002_WT.txt
- Location: GSE142277/GSM4224315/GSM4224315_out_gene_exon_tagged.dge_intronssf002_WT.txt


## Analysis

### Convert the data to AnnotatedData

In [ ]:
# data_manager = cdata.dataset_manager()
# exons_file = data_manager.get_file("GSE142277/GSM4224315/GSM4224315_out_gene_exon_tagged.dge_exonssf002_WT.txt")
# introns_file = data_manager.get_file("GSE142277/GSM4224315/GSM4224315_out_gene_exon_tagged.dge_intronssf002_WT.txt")
# exons = sc.read_csv(exons_file, delimiter = "\t").transpose()
# introns = sc.read_csv(introns_file, delimiter = "\t").transpose()
# adata = crna.tl.add_intron_data(exons, introns)
# #adata.write('/home/jovyan/external/GSE142277/GSM4224315/GSM4224315.h5ad')
# adata.write('/home/jovyan/notebooks/sysmo/cell-cycle/saved-data/GSM4224315.h5ad')

### Preprocess

In [ ]:
#adata = sc.read_h5ad("/home/jovyan/external/GSE142277/GSM4224315/GSM4224315.h5ad")
adata = sc.read_h5ad('/home/jovyan/notebooks/sysmo/cell-cycle/saved-data/GSM4224315.h5ad')
adata

In [ ]:
crna.qc.qc_plots(adata)

In [ ]:
# Filter cells according to identified QC thresholds:
print('Total number of cells: {:d}'.format(adata.n_obs))

sc.pp.filter_cells(adata, min_counts = 1500)
print('Number of cells after min count filter: {:d}'.format(adata.n_obs))

sc.pp.filter_cells(adata, max_counts = 40000)
print('Number of cells after max count filter: {:d}'.format(adata.n_obs))

adata = adata[adata.obs['mt_frac'] < 0.2]
print('Number of cells after MT filter: {:d}'.format(adata.n_obs))

sc.pp.filter_cells(adata, min_genes = 700)
print('Number of cells after gene filter: {:d}'.format(adata.n_obs))


In [ ]:
crna.qc.qc_plots(adata)

Save the original counts in separate layers and then:
 - filter 
 - normalize counts per cell
 - apply log transform
 - filter highly variable genes

In [ ]:
adata.layers['spliced_counts'] = adata.layers['spliced'].copy()
adata.layers['unspliced_counts'] = adata.layers['unspliced'].copy()
# sc.pp.filter_genes(adata, min_counts = 20)
# sc.pp.filter_genes(adata, min_cells = 5)
adata_all = adata.copy()
scv.pp.filter_and_normalize(
    adata, 
    min_counts = 20, min_cells = 5, log = True,
    n_top_genes = 3000, subset_highly_variable = True
)
# sc.pp.normalize_total(adata)
# sc.pp.log1p(adata)
# sc.pp.highly_variable_genes(adata, n_top_genes = 3000)
# adata = adata[:, adata.var['highly_variable']]

Apply PCA (50 components) and compute nearest neighbours

In [ ]:
sc.pp.pca(adata)
sc.pp.neighbors(adata, n_neighbors = 10, n_pcs = 50)

Compute reduced dimensional representation for visualization

In [ ]:
sc.tl.umap(adata, n_components = 3)
sc.tl.diffmap(adata)

We use the `scvelo` built in function `score_genes_cell_cycle` to compute the likelihood of each cell being in S-Phase or G2M-Phase, based on marker gene expression. ([Macosko et al.](https://www.cell.com/fulltext/S0092-8674(15)00549-8))

In [ ]:
scv.tl.score_genes_cell_cycle(adata)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize = (15, 5))
scv.pl.scatter(adata, color_gradients=['S_score', 'G2M_score'], basis = 'pca', smooth=True, perc=[5, 95], ax = axes[0], show = False)
axes[0].set_title("PCA")
scv.pl.scatter(adata, color_gradients=['S_score', 'G2M_score'], basis = 'umap', smooth=True, perc=[5, 95], ax = axes[1], show = False)
axes[1].set_title("UMAP")
scv.pl.scatter(adata, color_gradients=['S_score', 'G2M_score'], basis = 'diffmap', smooth=True, perc=[5, 95], ax = axes[2], show = False)
axes[2].set_title("DiffMap")

It seems like even the PCA method separates well the cells based on the cycle phase. Here is the PCA with an extra dimension.

In [ ]:
crna.pl.plot_scatter_3d(adata, color_gradients = ['S_score', 'G2M_score'])

As the authors of the original articles point out, the cell distribution can be approximated as a cylindrical surface in the space of the first 3 principle components. Unlike the autors, who analyze the data using cell cycle markers for determining the proper axis of the cylinder, we will try to analyze the manifold without using any prior knowledge about gene markers. For that person we will use the concept of RNA velocity as described in [Generalizing RNA velocity to transient cell states through dynamical modeling](https://www.nature.com/articles/s41587-020-0591-3)

## Computing RNA velocity

The `scvelo` library from Theiss lab is used to compute the gene dynamics

In [ ]:
scv.pp.moments(adata, n_pcs = 50, n_neighbors = 30)
scv.tl.recover_dynamics(adata, n_jobs = 8)
scv.tl.velocity(adata, mode='dynamical')
scv.tl.velocity_graph(adata)

In [ ]:
adata

Original: 1000 cells x 20000 genes
QC: 700 cells x 10000 genes
Normalizations....
Highly variable genes: 700 cells x 3000 genes
PCA: 700 cells x 50 components


In [ ]:
adata.obsm['X_pca'].shape

The velocity plot in 2D

In [ ]:
scv.pl.velocity_embedding(adata, color_gradients=['S_score', 'G2M_score'], basis = 'pca', arrow_length = 2, arrow_size = 1, dpi = 100)

And in 3D

In [ ]:
crna.pl.plot_arrows_3d(adata, arrows = 'velocity_pca', color_gradients = ['S_score', 'G2M_score'])

## Finding rotational parameters

A motion on a cylindrical surface around the axis of the cylinder can be described as:

$$V = W(X - X_0)$$

where $X_0$ shifts the cylinder axis to pass through the origin, and $W$ is an "angular velocity" matrix, whose columns are orthogonal, has 2 purely imaginary eigenvalues and the rest of the eigenvalues are 0.

Using tensorflow, we will approximate the matrix $W$ from the expression data and the RNA velocity:

In [ ]:
cdev.reload_user_libs(crna.tl)
x_v_fit = crna.tl.fit_matrix(adata.obsm['X_pca'], adata.obsm['velocity_pca'], n_comp = 50)

We compute the eigenvalues of the matrix W. As expected, the first 2 eigenvalues have significantly higher abolute value than the following ones. Also, they have small real values, and thus are nearly imaginary. The eigenvectors corresponding to these eigenvalues are complex conjugate of each other. Because multiplication by $j$ corresponds to rotation of 90 degrees, the real and imaginary part of each eigenvectors should define perpendicular vectors in the plane of rotation. In reality they are 'almost' normal, so we ensure orthogonality. Taking for granted from the original paper, that the axis of rotation lies in the subspace defined by the first 3 PCs, we define the axis of rotation as the plane normal in this 3D space. (maybe this assumption should be revised later...)

In [ ]:
W = x_v_fit['W']
scaling_fct = np.abs(np.linalg.eig(W)[0][0])
W = W / scaling_fct
e_val, e_vec = np.linalg.eig(W)
# print(e_val)
# print('Eigenvalues (arg):')
# print()
with pd.option_context('display.float_format', '{:,.3f}'.format):
    print('Eigenvalues')
    display(pd.DataFrame(
        {'Eigenvalues': e_val, 'Eigenvalues (arg)': np.angle(e_val) * 180 / np.pi,
         'Eigenvalues (abs)': np.abs(e_val)}).iloc[:20, :].T)
    print('First eigenvectors')
    display(pd.DataFrame(e_vec[:20, :20]).T)
# print(e_vec[:, 0])
# print(e_vec[:, 1])
plt.plot(np.abs(e_val), '.')
plt.legend('Eigenvalues (abs):')

dc1_dir = np.real(e_vec[:, 0])
dc1_dir /= np.linalg.norm(dc1_dir)
dc2_dir = np.imag(e_vec[:, 0])
angle_r_i = np.arccos(np.dot(dc1_dir, dc2_dir)/(np.linalg.norm(dc1_dir) * np.linalg.norm(dc2_dir))) * 180 / np.pi
print("Angle between real and imag: ", angle_r_i)
dc2_dir -= dc1_dir * np.dot(dc1_dir, dc2_dir)
dc2_dir /= np.linalg.norm(dc2_dir)
axis_dir = scipy.linalg.null_space([dc1_dir, dc2_dir]).flatten()
axis_dir /= np.linalg.norm(axis_dir)
#print()
plt.figure()
plt.plot(dc1_dir, '.', label = 'DC1')
plt.plot(dc2_dir, '.', label = 'DC2')
plt.legend()
# print(np.linalg.norm(dc1_dir))
# print('DC2')
# print(dc2_dir)
# print(np.linalg.norm(dc2_dir))
# print('Axis vector')
# print(axis_dir)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize = (15, 5))
hm0 = axes[0].imshow(x_v_fit['W'])
axes[0].set_title('W = V \\ X')
fig.colorbar(hm0, ax = axes[0])
hm2 = axes[1].imshow([np.abs(e_val)])
fig.colorbar(hm2, ax = axes[1])
axes[1].set_title('Eigenvalues (abs)')
pass

Adding the axes to the RNA velocity plot shows the computed axes of rotation (in orange): and it seems about right

In [ ]:
crna.pl.plot_arrows_3d(adata, arrows = 'velocity_pca', color_gradients = ['S_score', 'G2M_score'], directions = [10 * axis_dir, 10 * dc1_dir, 10 * dc2_dir])

Let's transform all the expression and velocity data from the PCA space to the space defined by the rotation: DC space (dynamic components). Also we transform the coordinates of each cell into polar coordinates, labeling the radius as `pseudo_r` and the angle, normalized to the interval 0-1 as `pseudo_t`. Our claim is that the angle represents the time along the cell cycle. (The offset is adjusted as explained later so that mitosis is at $t = 0$)

In [ ]:
def project_into_plane(adata, plane_vecs_col):
    n_comp = plane_vecs_col.shape[0]
    X_DC = np.dot(adata.obsm['X_pca'][:, :n_comp], plane_vecs_col)
    V_DC = np.dot(adata.obsm['velocity_pca'][:, :n_comp], plane_vecs_col)
    # Here the transformation to pseudotime should be adjusted
    adata.obs['pseudo_t'] = np.mod(np.arctan2(X_DC[:, 1], X_DC[:, 0]) / np.pi / 2, 1)
    adata.obs['pseudo_r'] = np.sqrt(np.power(X_DC[:, 1], 2) + np.power(X_DC[:, 0], 2))
    adata.obs['dc1'] = X_DC[:, 0]
    adata.obs['dc2'] = X_DC[:, 1]
    adata.obsm['X_dc'] = X_DC
    adata.obsm['velocity_dc'] = V_DC
    adata.varm['DCs'] = np.dot(adata.varm['PCs'], plane_vecs_col)
    
project_into_plane(adata, np.array([dc1_dir, dc2_dir]).T)

Now we can visualize the cell cycle well also in 2D:

In [ ]:
fig, axes = plt.subplots(1, 2, figsize = (15, 5))
scv.pl.velocity_embedding(adata, color = 'phase', basis = 'dc', arrow_length = 4, arrow_size = 2, legend_loc = 'on data', ax = axes[0], show = False)
axes[0].scatter(0, 0, marker = '+', c = 'k')
scv.pl.velocity_embedding_grid(adata, color = 'phase', basis = 'dc', arrow_length = 4, arrow_size = 2, legend_loc = 'on data', ax = axes[1], show = False)
axes[1].scatter(0, 0, marker = '+', c = 'k')

Let's plot the cell phase scores, computed previously and the phase assigned along the pseudo time. The phases seem well separated, which validates our assumption that this circular motion represents the cell cycle

In [ ]:
fig, axes = plt.subplots(1, 2, figsize = (15, 5))
sc.pl.scatter(adata, x = 'pseudo_t', y = 'S_score', color = 'phase', ax = axes[0], show = False, legend_loc= 'on data')
sc.pl.scatter(adata, x = 'pseudo_t', y = 'G2M_score', color = 'phase', ax = axes[1], show = False, legend_loc= 'on data')

The values of the dynamical components can also be plotted along the cell cycle:

In [ ]:
fig, axes = plt.subplots(1, 2, figsize = (15, 5))
#sc.pl.scatter(adata, x = 'pseudo_t', y = 'pseudo_r', color = 'phase', legend_loc= 'on data', ax = axes[0, 0], show = False)
sc.pl.scatter(adata, x = 'pseudo_t', y = 'dc1', color = 'phase', legend_loc= 'on data', ax = axes[0], show = False)
sc.pl.scatter(adata, x = 'pseudo_t', y = 'dc2', color = 'phase', legend_loc= 'on data', ax = axes[1], show = False)
#sc.pl.scatter(adata, x = 'pseudo_t', y = 'dc3', color = 'phase', legend_loc= 'on data', ax = axes[2], show = False)

In [ ]:
# def save_results(dry_run = False):
#     # Order observation in pseudotime order
#     time_order = np.argsort(adata.obs['pseudo_t'])
#     adata_ord = adata[time_order, :]
#     adata_all_ord = adata_all[time_order, :]
#     # Update adata_all with computed data
#     adata_all_ord.obs = adata_ord.obs[['n_counts', 'n_genes', 'n_counts_unspliced', 'n_genes_unspliced', 'mt_frac', 'S_score', 'G2M_score', 'phase', 'pseudo_t']].copy()
#     # Extract only the basic information to save
#     adata_save = sc.AnnData(
#         X = adata_ord.X, obs = adata_ord.obs, var = adata_ord.var, 
#         obsm = adata_ord.obsm, varm = adata_ord.varm,
#         layers = adata_ord.layers
#     )
#     print(adata_save)
#     if not dry_run:
#         adata_save.write('/home/jovyan/notebooks/sysmo/cell-cycle/saved-data/GSM4224315_qc_velocity_t.h5ad')
#     adata_all_save = sc.AnnData(X = adata_all_ord.X, obs = adata_all_ord.obs, var = adata_all_ord.var, layers = adata_all_ord.layers)
#     print(adata_all_save)
#     if not dry_run:
#         adata_all_save.write('/home/jovyan/notebooks/sysmo/cell-cycle/saved-data/GSM4224315_qc_all_genes_counts_t.h5ad')
# save_results()


Now let's use the order determined by `pseudo_t` and plot the absolute number of spliced and unspliced reads for each cell (normalized to 1e4 and 1e3 respectively and then smoothed). We expect that:
  - the number grows from G1 to M
  - the number drops sharply in the M phase
  - the ratio of spliced to unspliced RNA drops during the M phase, due to chromosme packing and inactivation

We see that these predictions hold, and also the second one helps us identify better the moment of mitosis (the black line)

In [ ]:
import scipy.signal as sig
n_spliced_raw = np.sum(adata.layers['spliced_counts'], axis = 1, keepdims = True) / 1e4
n_unspliced_raw = np.sum(adata.layers['unspliced_counts'], axis = 1, keepdims = True) / 1e3
time_order = np.argsort(adata.obs['pseudo_t'])
t, n_spliced = crna.tools.bin_smooth(
    adata.obs['pseudo_t'][time_order], 
    n_spliced_raw[time_order],
    n_bins = 100
)
_, n_unspliced = crna.tools.bin_smooth(
    adata.obs['pseudo_t'][time_order], 
    n_unspliced_raw[time_order],
    n_bins = 100
)
n_spliced = n_spliced.flatten()
n_spliced = np.hstack([n_spliced, n_spliced])
n_spliced = sig.savgol_filter(n_spliced, 21, 3)
n_unspliced = n_unspliced.flatten()
n_unspliced = np.hstack([n_unspliced, n_unspliced])
n_unspliced = sig.savgol_filter(n_unspliced, 31, 3)
# plt.scatter(adata.obs['pseudo_t'], n_spliced_raw, s = 2, c = 'b', label = 'spliced')
# plt.scatter(adata.obs['pseudo_t'], n_unspliced_raw, s = 2, c = 'r', label = 'unspliced')
plt.plot(np.hstack([t,t + 1]), n_spliced, 'b', label = 'spliced')
plt.plot(np.hstack([t,t + 1]), n_unspliced, 'r', label = 'unspliced')
plt.plot([1, 1], [0, 0.5], 'k')
plt.legend()
pass

Let's have a look at the cyclin levels. Unfortunately CCND3 is not a highly variable gene, so only the other 3 can be plotted:

## (Preliminary) Gene Analysis

In [ ]:
adata_all.obs['pseudo_t'] = adata.obs['pseudo_t'].copy()
scv.pl.heatmap(adata, var_names = ['CCND3', 'CCNE2', 'CCNA2', 'CCNB1'], sortby = 'pseudo_t', n_convolve = 10, colorbar = True, sort = False)
#scv.pl.heatmap(adata_all, var_names = ['CCND3', 'CCNE2', 'CCNA2', 'CCNB1'], sortby = 'pseudo_t', n_convolve = 10, colorbar = True, sort = False)

The absolute counts are rather low, so for most ones the signal to noise ratio is rather small.

In [ ]:
scv.pl.scatter(adata_all, x = 'pseudo_t', y = ['CCND3', 'CCNE2', 'CCNA2', 'CCNB1'])

In [ ]:
def get_high_weight_genes(adata, pc_index, thresholds):
    pc1_weights = adata.varm['PCs'][:, pc_index]
    pc1_order = np.argsort(pc1_weights)
    h_genes = pc1_weights > thresholds['pos']
    print(np.sum(h_genes))
    l_genes = pc1_weights < thresholds['neg']
    print(np.sum(l_genes))
    print(adata.var.index.values[h_genes | l_genes])
    #plt.plot(pc1_weights[pc1_order], '.')
    
get_high_weight_genes(adata, 0, thresholds = {'pos': 0.01, 'neg': -0.1})
get_high_weight_genes(adata, 1, thresholds = {'pos': 0.1, 'neg': -0.1})
get_high_weight_genes(adata, 2, thresholds = {'pos': 0.1, 'neg': -0.1})
get_high_weight_genes(adata, 3, thresholds = {'pos': 0.1, 'neg': -0.1})

In [ ]:
pc_genes = ['ARL6IP1', 'ASPM', 'AURKA', 'C12orf45', 'CCNB1', 'CCNE2', 'CDCA7', 'CDK1', 'CENPF', 'CKS2', 'CTC-338M12.4', 'FAM111B', 'GINS2', 'HIST1H2AC', 'HIST1H2BD', 'HIST3H2A', 'HMGB2', 'KIF23', 'KPNA2', 'KRT18', 'MKI67', 'MSH6', 'NR2C2AP', 'NUSAP1', 'PCNA', 'PRC1', 'RPLP1', 'SMC4', 'TOP2A', 'TPX2', 'UBE2C', 'UNG', 'AURKA', 'CCNB1', 'CDC20', 'CKS2', 'MT-ATP6', 'MT-CO1', 'MT-CO2', 'MT-CO3', 'MT-CYB', 'MT-ND1', 'MT-ND2', 'MT-ND4', 'MT-ND4L', 'MT-ND5', 'MT-RNR1', 'TOP2A', 'TPX2', 'TUBA1C', 'UBE2C', 'ARL6IP1', 'CCNB1', 'CDK1', 'DYNLL1', 'GINS2', 'GMNN', 'HIST1H4C', 'HMGB2'
 'KIAA0101', 'NTS', 'PCNA', 'PTTG1', 'RRM2', 'SNHG3', 'TYMS', 'UBE2T', 'USP1']
pc_genes = set([g for g in pc_genes if not g.startswith('MT-')])
scv.pl.heatmap(adata, var_names = pc_genes, sortby = 'pseudo_t', n_convolve = 30, colorbar = True, figsize = (15, 15))


In [ ]:
adata

In [ ]:
def test_transformation(adata):
    gene = 'PCNA'
    time_order = np.argsort(adata.obs['pseudo_t'])
    X0 = adata[time_order, gene].X.flatten()
    X0 = X0 - np.mean(X0)
    plt.plot(X0, '.')
    gene_weights_pca = adata.varm['PCs']
    gene_weights_dca = adata.varm['DCs']
    gene_pc = adata.obsm['X_pca']
    gene_dc = adata.obsm['X_dc']
    print(gene_weights_pca.shape)
    print(gene_pc.shape)
    adata.layers['PCA_expr'] = np.dot(gene_pc[:, :2], gene_weights_pca[:, :2].T)
    adata.layers['DCA_expr'] = np.dot(gene_dc, gene_weights_dca.T)
    plt.plot(adata[time_order, gene].layers['PCA_expr'].flatten(), '.')
    plt.plot(adata[time_order, gene].layers['DCA_expr'].flatten(), '.')
    
test_transformation(adata)
